📌 何をしていたか？ (目的)

あなたの研究の出発点となる**「データ論文」そのもののマスターリストを作成**していました。

「データ論文」とは、データセットを主たる成果物として報告する論文のことです。

✅ どこまで行っているか？ (達成状況)

Scopus API に「DOCTYPE(dp)（データペーパー）」という条件で問い合わせを行いました。

ヒットした全データ論文（約 19,000 件）の基本情報（EID, DOI, タイトル, 発行年, 被引用数）を全て収集しました。

収集した全情報を data/processed/data_papers.csv という CSV ファイルに保存済みです。

➡️ このノートブックの成果物:

data_papers.csv: 後続のすべての分析の基礎となる、データ論文の完全なリスト。


In [ ]:
import requests
import pandas as pd
import time
import os
from urllib.parse import urlparse, parse_qs
from tqdm import tqdm

# --- 設定項目 ---

# ご自身のAPIキー
API_KEY = "90469972beed34fa2913dc1ad6a644ac" 

# 検索クエリ（データペーパーを指定）
QUERY = "DOCTYPE(dp)"

# 保存先のディレクトリとファイル名
OUTPUT_DIR = '../data/processed'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'data_papers.csv')

# APIリクエストの基本URL
BASE_URL = "https://api.elsevier.com/content/search/scopus"

# --- 事前チェック：総件数の取得 ---
print("データ論文の総件数を取得しています...")
try:
    pre_params = {'apiKey': API_KEY, 'query': QUERY, 'count': 1}
    pre_response = requests.get(BASE_URL, params=pre_params)
    pre_response.raise_for_status()
    total_results = int(pre_response.json().get('search-results', {}).get('opensearch:totalResults', 0))
    print(f"収集対象のデータ論文は全 {total_results} 件です。")
except requests.exceptions.RequestException as e:
    print(f"総件数の取得中にエラーが発生しました: {e}")
    total_results = 0

# --- データ収集処理 ---

# 保存する論文情報を格納するリスト
all_papers_data = []

# プログレスバーの初期化
pbar = tqdm(total=total_results, desc="データ論文収集中")

if total_results > 0:
    # 最初のAPIリクエストのパラメータ
    params = {
        'apiKey': API_KEY,
        'query': QUERY,
        'cursor': '*',
        'count': 25,
        'view': 'STANDARD'
    }

    try:
        while 'cursor' in params and params['cursor']:
            # APIリクエストを実行
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status()

            data = response.json()
            results = data.get('search-results', {})
            entries = results.get('entry', [])

            if not entries:
                break
            
            # 必要な情報を抽出してリストに追加
            for entry in entries:
                all_papers_data.append({
                    'eid': entry.get('eid', ''),
                    'doi': entry.get('prism:doi'),
                    'title': entry.get('dc:title'),
                    'publication_year': entry.get('prism:coverDate', '')[:4],
                    'citedby_count': entry.get('citedby-count')
                })
            
            # プログレスバーを更新
            pbar.update(len(entries))

            # 次のカーソル情報を探して更新
            next_cursor_url = None
            for link in results.get('link', []):
                if link.get('@ref') == 'next':
                    next_cursor_url = link.get('@href')
                    break
            
            if next_cursor_url:
                parsed_url = urlparse(next_cursor_url)
                query_params = parse_qs(parsed_url.query)
                params['cursor'] = query_params.get('cursor', [None])[0]
            else:
                break

            time.sleep(1) # サーバーへの配慮

    except requests.exceptions.RequestException as e:
        print(f"\nAPIリクエスト中にエラーが発生しました: {e}")
    except Exception as e:
        print(f"\n予期せぬエラーが発生しました: {e}")
    finally:
        # ループが終了したらプログレスバーを閉じる
        pbar.close()

# --- データの保存 ---
if all_papers_data:
    # 収集した件数が総件数と異なる場合、メッセージを表示
    if len(all_papers_data) < total_results:
        print(f"\n注意: 処理が途中で終了した可能性があります。{total_results}件中 {len(all_papers_data)}件を収集しました。")

    df = pd.DataFrame(all_papers_data)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"\n処理完了。合計 {len(df)} 件のデータ論文を '{OUTPUT_FILE}' に保存しました。")
    print("\n--- 保存されたデータの先頭5件 ---")
    print(df.head())
else:
    print("収集できたデータがありませんでした。")

データ論文の収集を開始します...
現在までに収集した論文数: 25
現在までに収集した論文数: 50
現在までに収集した論文数: 75
現在までに収集した論文数: 100
現在までに収集した論文数: 125
現在までに収集した論文数: 150
現在までに収集した論文数: 175
現在までに収集した論文数: 200
現在までに収集した論文数: 225
現在までに収集した論文数: 250
現在までに収集した論文数: 275
現在までに収集した論文数: 300
現在までに収集した論文数: 325
現在までに収集した論文数: 350
現在までに収集した論文数: 375
現在までに収集した論文数: 400
現在までに収集した論文数: 425
現在までに収集した論文数: 450
現在までに収集した論文数: 475
現在までに収集した論文数: 500
現在までに収集した論文数: 525
現在までに収集した論文数: 550
現在までに収集した論文数: 575
現在までに収集した論文数: 600
現在までに収集した論文数: 625
現在までに収集した論文数: 650
現在までに収集した論文数: 675
現在までに収集した論文数: 700
現在までに収集した論文数: 725
現在までに収集した論文数: 750
現在までに収集した論文数: 775
現在までに収集した論文数: 800
現在までに収集した論文数: 825
現在までに収集した論文数: 850
現在までに収集した論文数: 875
現在までに収集した論文数: 900
現在までに収集した論文数: 925
現在までに収集した論文数: 950
現在までに収集した論文数: 975
現在までに収集した論文数: 1000
現在までに収集した論文数: 1025
現在までに収集した論文数: 1050
現在までに収集した論文数: 1075
現在までに収集した論文数: 1100
現在までに収集した論文数: 1125
現在までに収集した論文数: 1150
現在までに収集した論文数: 1175
現在までに収集した論文数: 1200
現在までに収集した論文数: 1225
現在までに収集した論文数: 1250
現在までに収集した論文数: 1275
現在までに収集した論文数: 1300
現在までに収集した論文数: 1325
現在までに収集した論文数: 135